<a href="https://colab.research.google.com/github/yongsa-nut/TU671_CN408_GenAI/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 Count Tokens (1 point)

Count the number of tokens using [gpt-4 tokenizer](https://platform.openai.com/tokenizer) (you can use the web or the code) of the following four prompts:

- The old man sat on the park bench, feeding pigeons and lost in thought. As the sun began to set, he smiled, realizing that sometimes the simplest moments are the most precious.

- ชายชรานั่งอยู่บนม้านั่งในสวนสาธารณะ ให้อาหารนกพิราบและจมอยู่ในภวังค์ความคิด เม่ือดวงอาทิตย์เรี่มตกดิน เขายิ้ม ตระหนักว่าบางครั้งช่วงเวลาที่เรียบง่ายที่สุดกลับมีค่าที่สุด

- 老人は公園のベンチに座り、鳩に餌をやりながら物思いにふけっていた。日が沈み始めると、彼は微笑み、時として最も単純な瞬間が最も貴重であることに気づいた。

- 9.11 > 9.9

In [ ]:
# List your answers here


## 2) Word Embedding: Closet words (1 point)

For this question, your job is find the closest words of the following words in the vector space of Word2Vec embedding.

1) cat

2) King

3) teaching

4) learning

The code is provided below.

In [ ]:
# First install the library spacy
!pip install gensim

In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors
import numpy as np

# Load pre-trained Word2Vec embeddings (~1.6gb)
model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Example of usage
word = 'Merry'
model.similar_by_word(word) # It will return top 10 closest words. ['word','cosine similarity']

[('Shavaun', 0.49784329533576965),
 ('Apple_Dumpling', 0.4907097816467285),
 ('Merry_Merry', 0.4760850667953491),
 ('Christma', 0.4625037908554077),
 ('Gravener', 0.4579834043979645),
 ('OPEN_RESTRICTED_1', 0.45711594820022583),
 ('Reindeers', 0.4541266858577728),
 ('Rub_Dub_Dub', 0.4538874328136444),
 ('Fancy_Nancy_Splendiferous', 0.4518665373325348),
 ('XMas', 0.45152536034584045)]

In [ ]:
# List your answers here

## 3) Word Embedding: Analogy (1 point)

For this question, you task is to find four analogies of the following form:

**man** is to **woman** as ***king*** is to ***queen***.

Specifically, your job is to come up with the first three bold words (man, woman, king) and the code below will generate the analogy word (queen) using Word2Vec as discussed in class. See code below for more examples.

In [ ]:
# Need to run the code to load up the model first from 2)
def vector_analogy(word1, word2, word3):
    try:
        # Get word vectors
        vec1 = model[word1]
        vec2 = model[word2]
        vec3 = model[word3]

        # Compute the analogy vector
        analogy_vec = vec2 - vec1 + vec3

        # Find the most similar word to the analogy vector
        result = model.similar_by_vector(analogy_vec, topn=5)

        # Filter out input words
        for word, similarity in result:
            if word not in [word1, word2, word3]:
                return word

        return "No suitable word found"
    except KeyError:
        return "One or more words not in vocabulary"

# Example usage
analogies = [
    ("man", "woman", "king"),
    ("paris", "france", "rome"),
    ("big", "bigger", "small"),
    ("japan", "sushi", "italy")
]

for w1, w2, w3 in analogies:
    result = vector_analogy(w1, w2, w3)
    print(f"{w1} is to {w2} as {w3} is to {result}")

man is to woman as king is to queen
paris is to france as rome is to italy
big is to bigger as small is to larger
japan is to sushi as italy is to gourmet_pizza


In [ ]:
# List your answers here the same as the example code above.
analogies = [
    ()
]

## 4) Sentence Embedding: Closest sentences (2 points)

For this question, you will be using cohere API to explore Sentence Embedding. The idea is still the same, similar sentences will be closer in the embedding space.

To run the code below, the first step you must obtain cohere API trial (free) key from their [website](https://dashboard.cohere.ai/welcome/register).

![key_trial.png](https://drive.google.com/uc?export=view&id=1rdMaMdO5S7eaktN0qwodhgQd3auuZ6gi)

* Note: Be careful about rate limit for free key. 5 calls per minute.

In [ ]:
!pip install cohere

In [ ]:
import numpy as np
import cohere
co = cohere.Client("") # Your Cohere API key

In [ ]:
# This is our documents. It is just a list of strings (sentences)
# (Taken from https://github.com/anthropics/anthropic-cookbook/blob/main/third_party/VoyageAI/how_to_create_embeddings.md)
documents = [
    "The Mediterranean diet emphasizes fish, olive oil, and vegetables, believed to reduce chronic diseases.",
    "Photosynthesis in plants converts light energy into glucose and produces essential oxygen.",
    "20th-century innovations, from radios to smartphones, centered on electronic advancements.",
    "Rivers provide water, irrigation, and habitat for aquatic species, vital for ecosystems.",
    "Apple’s conference call to discuss fourth fiscal quarter results and business updates is scheduled for Thursday, November 2, 2023 at 2:00 p.m. PT / 5:00 p.m. ET.",
    "Shakespeare's works, like 'Hamlet' and 'A Midsummer Night's Dream,' endure in literature."
]

# Embedded (More info: https://docs.cohere.com/reference/embed?ref=cohere-ai.ghost.io )
response = co.embed(
    texts=documents,
    model='embed-english-v3.0',
    input_type='search_document'
)

embeddings = response.embeddings

print(f"The dimension of the embedding =  {len(embeddings[0])}")

print(f"Embedding for sentence 1: {embeddings[0]}")
print(f"Embedding for sentence 2: {embeddings[1]}")
print(f"Embedding for sentence 3: {embeddings[0]}")

# To calculate similarity we can use dot product (same as cosine similarity when vectors have been normarlized to 1.)
print("Similarity between sentences 1 and 2:", np.dot(embeddings[0], embeddings[1]))
print("Similarity between sentences 1 and 3:", np.dot(embeddings[0], embeddings[2]))
print("Similarity between sentences 2 and 3:", np.dot(embeddings[0], embeddings[3]))

# A similar sentence to the first sentence
temp = "Focusing on seafood, olive oil, and plant-based foods, the Mediterranean eating pattern is thought to lower the risk of long-term health conditions."
new_embeddings = co.embed(
    texts=[temp],
    model='embed-english-v3.0',
    input_type='search_document'
)
new_embeddings = new_embeddings.embeddings

print("Similarity between sentences 1 and the paraphased version:",
      np.dot(embeddings[0], new_embeddings[0]))

In [ ]:
# Helper function for question 4
def closest_sentence(sentence, documents, model='embed-english-v3.0'):
  # Merge into one so that it would only use one call to API.
  documents = documents.copy() + [sentence]
  response = co.embed(
    texts=documents,
    model=model,
    input_type='search_document'
  )
  embeddings = response.embeddings
  similarities = [np.dot(embeddings[i], embeddings[-1]) for i in range(len(embeddings)-1)]
  # print similarities and index
  for i, sim in enumerate(similarities):
    print(f"Similarity between sentence {i+1} and the new sentence: {sim}")

  max_index = np.argmax(similarities)
  return max_index, documents[max_index]

In [ ]:
# 4.1) Find a sentence that is closest to the fifth sentence ("Apple’s conference call ...")
your_sentence = "" #Your sentence here
index, sentence = closest_sentence(your_sentence, documents)
print(f"The closest sentence to '{your_sentence}' is '{sentence}' at index {index}.")

In [ ]:
# 4.2) Find a sentence that is closest to the sixth sentence ("Shakespeare's works ... ")
#     The sentence must not contain the word 'Shakespeare'
your_sentence = "" #Your sentence here
index, sentence = closest_sentence(your_sentence, documents)
print(f"The closest sentence to '{your_sentence}' is '{sentence}' at index {index}.")

In [ ]:
# 4.3) Find a sentence in "Thai" that is closest to the second sentence. ("Photosynthesis in plants converts ...")
your_sentence = "" #Your sentence here
index, sentence = closest_sentence(your_sentence, documents, 'embed-multilingual-v3.0')
print(f"The closest sentence to '{your_sentence}' is '{sentence}' at index {index}.")

In [ ]:
# 4.4) Find a sentence in "Japan" that is closest to the first sentence. ("The Mediterranean  ...")
your_sentence = "" #Your sentence here
index, sentence = closest_sentence(your_sentence, documents, 'embed-multilingual-v3.0')
print(f"The closest sentence to '{your_sentence}' is '{sentence}' at index {index}.")